In [38]:
# Purpose:  Model topics for group 5 ( 9th republic debate and 6th dem debate)
# Author:  Preeti Javaji
# Date:  05/17/16
# Course:  MAS DSE capstone, Spring 2016

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import math

pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


print 'done'

done


In [40]:
# Function defs.

# return string with non-ascii chars removed
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

print 'done'

done


In [41]:
# Load data file & clean the 'text' column.

df = pd.read_csv("Sentiment_backup.csv")

# Convert to ascii
df['ascii'] = df['text'].apply(to_ascii)
#df['ascii'] = df['ascii'].str.encode('ascii',errors='ignore') 
df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
df['ascii'] = df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
df['ascii'] = df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
df['ascii'] = df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
df['ascii'] = df['ascii'].str.lower()

df['K_sentiment'] = df['sentiment']
print df.shape
#print df[df['ascii']!=df['text']][['ascii', 'K_sentiment']].head(10)
df[['text', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

(4333, 23)


,text,ascii
0,RT @TheBaxterBean: Scott Walker's Abortion Ban Allows Rapist Father To Sue For Emotional Distress http://t.co/rHMvgumuir #GOPDebate @Badger��_,rt @thebaxterbean scott walkers abortion ban allows rapist father to sue for emotional distress url #gopdebate @badger_
1,"RT @mydaughtersarmy: Dear GOP,\rIf you want to stop abortions... \r\r#GOPDebate \rhttp://t.co/RbL1t4iPx6","rt @mydaughtersarmy dear gop,\rif you want to stop abortions... \r\r#gopdebate \rurl"
2,RT @Bipartisanism: A message to the #GOPDebate candidates: http://t.co/WFK5CmVgiN,rt @bipartisanism a message to the #gopdebate candidates url


In [42]:
# Show the class frequencies.
counts = df['subject_matter'].value_counts()
total = counts.sum()

counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )

#print("%.2f" % a)
#string = 'string%d' % (i,)

print counts
print type(counts)

None of the above                       (Count: 1271, Frequency: 0.33)
Religion                                 (Count: 407, Frequency: 0.11)
Women's Issues (not abortion though)     (Count: 362, Frequency: 0.10)
Racial issues                            (Count: 353, Frequency: 0.09)
Abortion                                 (Count: 293, Frequency: 0.08)
Jobs and Economy                         (Count: 251, Frequency: 0.07)
Immigration                              (Count: 211, Frequency: 0.06)
FOX News or Moderators                   (Count: 188, Frequency: 0.05)
LGBT issues                              (Count: 126, Frequency: 0.03)
iran deal                                 (Count: 74, Frequency: 0.02)
Healthcare (including Medicare)           (Count: 67, Frequency: 0.02)
Gun Control                               (Count: 61, Frequency: 0.02)
Foreign Policy                            (Count: 41, Frequency: 0.01)
ISIS                                      (Count: 40, Frequency: 0.01)
none  

In [43]:
%pwd

u'/Users/preetijavaji/Documents/DSE/capstone'

In [44]:
# Add the 1st democratic debate transcript
D1_df = pd.read_csv("labeled_transcripts/R11_proc_tagged.csv")
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,238,KELLY,moderator,0,"KELLY: Good evening, and welcome to the fabulous FOX Theatre in downtown Detroit, the site of the 11th Republican presidential debate of the 2016 campaign. KELLY: I'm Megyn Kelly, along with my co-moderators, Bret Baier and Chris Wallace.",intro
1,1,408,BAIER,moderator,238,"BAIER: 59 Republican delegates are at stake here in the state of Michigan during next Tuesday's Republican primary, the biggest prize out of four states holding contests that day. For tonight's debate we're partnering with Facebook. The...",intro
2,2,50,CHEERING,audience,924,(CHEERING),none
3,3,50,APPLAUSE,audience,924,(APPLAUSE),none
4,4,50,CHEERING,audience,949,(CHEERING),none


In [45]:
#clean it up too
D1_df['ascii'] = D1_df['text'].apply(to_ascii)

D1_df['ascii'] = D1_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D1_df['ascii'] = D1_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D1_df['ascii'] = D1_df['ascii'].str.lower()

In [46]:
# Now take the text and topic from both the dataframes and concat
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})

In [47]:
# read the next debate transcript
D2_df = pd.read_csv("labeled_transcripts/D7_proc_labeled.csv")
D2_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,50,APPLAUSE,audience,587,[applause],intro
1,1,50,APPLAUSE,audience,645,[applause],intro
2,2,50,APPLAUSE,audience,729,[applause],intro
3,3,50,APPLAUSE,audience,1299,[applause],intro
4,4,3088,COOPER,moderator,0,"COOPER: And welcome to The Whiting Auditorium on the campus of the Flint Cultural Center in Flint, Michigan for a special CNN democratic presidential debate. I'm Anderson Cooper. I want to welcome our viewers in the United States and ar...",intro


In [48]:
#clean it up too
D2_df['ascii'] = D2_df['text'].apply(to_ascii)

D2_df['ascii'] = D2_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D2_df['ascii'] = D2_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D2_df['ascii'] = D2_df['ascii'].str.lower()

In [49]:
# Now take the text and topic from both the dataframes and concat
D2_new = D2_df[['ascii','topic']]

In [50]:
# read the next debate transcript
D3_df = pd.read_csv("labeled_transcripts/D8_proc_labeled.csv")
D3_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,28,RAMOS,moderator,0,RAMOS: [Speaking in Spanish],intro
1,1,210,SALINAS,moderator,28,"SALINAS [through translator]: This will be the first and only debate the candidates will do, taking into account the millions of [inaudible] voters [inaudible] Univision News, together with the Washington Post.",intro
2,2,140,RAMOS,moderator,238,"RAMOS [through translator]: Here with us tonight is Karen Tumulty, Washington Post correspondent [inaudible] specialized in national policy.",intro
3,3,50,APPLAUSE,audience,550,[applause],intro
4,4,50,APPLAUSE,audience,591,[applause],intro


In [51]:
#clean it up too
D3_df['ascii'] = D3_df['text'].apply(to_ascii)

D3_df['ascii'] = D3_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D3_df['ascii'] = D3_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D3_df['ascii'] = D3_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D3_df['ascii'] = D3_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D3_df['ascii'] = D3_df['ascii'].str.lower()
D3_new = D3_df[['ascii','topic']]

In [52]:
# read stuff from hashtags .csv
D4_df = pd.read_csv("labeled_transcripts/hashtagTweets_updated_1.csv")
D4_df.tail()

,topic,tweetid,text
1546,supremecourt,7.067650e+17,"""Possible #SupremeCourt Pick Championed Black History Museum #President #BarackObama #RobertL.Wilkins... https://t.co/kZwQyiHh1M """
1547,supremecourt,7.067620e+17,"""Tell @tedcruz: stop obstructing @POTUS #SCOTUS nominee. #doyourjob. Sign the petition https://t.co/Fw5l3eCs3l via @CREDOMobile #SupremeCourt """
1548,supremecourt,7.066720e+17,"""Obama has the right to nominate the next #SupremeCourt Justice. #DemDebate #ImWithHer #HillaryClinton """
1549,supremecourt,7.066670e+17,"""Joe #Biden jokingly suggests #Obama should nominate Ted #Cruz to the #SupremeCourt https://t.co/YoMYoHz0vY via @TheWeek Typical Biden _���_��� """
1550,supremecourt,7.066190e+17,"""#Biden To #Obama: ���If You Want To Remake The #SupremeCourt, Here�۪s What You Can Do��_�۪ https://t.co/xNXFMMWPPa - #POTUS #SCOTUS #DoYourJob """


In [53]:
#clean it up too
D4_df['ascii'] = D4_df['text'].apply(to_ascii)

D4_df['ascii'] = D4_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D4_df['ascii'] = D4_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D4_df['ascii'] = D4_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D4_df['ascii'] = D4_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D4_df['ascii'] = D4_df['ascii'].str.lower()

In [54]:
# Now concat all 4
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})
D2_new = D2_df[['ascii','topic']]
T_new1 = pd.concat([T_DF,D1_new])
print T_new1.shape
D3_new = D3_df[['ascii','topic']]
T_new2 = pd.concat([T_new1,D2_new])
print T_new2.shape
D4_new = D4_df[['ascii','topic']]
T_new3 = pd.concat([T_new2,D3_new])
print T_new3.shape
T_new = pd.concat([T_new3,D4_new])
print T_new.shape

(4951, 2)
(5307, 2)
(5678, 2)
(7229, 2)


In [55]:
# clean the topics before encoding
# first convert to lower case
T_new['topic'] = T_new['topic'].str.lower()
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 ' borders',
 'abortion',
 'afghanistan',
 'auto bailout',
 'benghazi',
 'bidding procedures',
 'bipartisanship',
 'black incarceration',
 'border',
 'border security',
 'bordersecurity',
 'buzzfeed tapes',
 'campaign finance',
 'candidate behavior',
 'character',
 'climate change',
 'clinton emails',
 'clintons emails',
 'clintons wall street speeches',
 'closing',
 'cnn',
 'cnndebate',
 'cnntownhall',
 'college',
 'college tuition',
 'common core',
 'community',
 'conservatism',
 'corporate america',
 'crumbling infrastructure',
 'cuba',
 'currency',
 'cutting government',
 'deathpenalty',
 'deficit',
 'delegate count',
 'economy',
 'education',
 'environment',
 'experience',
 'exporting jobs',
 'family leave',
 'familyleave',
 'flint water',
 'flintwater',
 'flintwatercrisis',
 'foreign aid',
 'foreign policy',
 'fox news or moderators',
 'fracking',
 'gay marriage',
 'gop',
 'gun control',
 'healthcare',
 'healthcare (including medicare)',
 'hillary',
 'homelandsecurity',
 'i

In [ ]:
# After seeing the list above we will combine:
1.'clinton emails','clintons emails',
2.'cnn','cnndebate','cnntownhall',
3.'college','college tuition', 'education','public education', 'schools',
4.'environment', 'fracking','climate change',
5.'family leave','familyleave',
6.'flint water','flintwater','flintwatercrisis',
7.'foreign aid', 'north korea','foreign policy', 'homelandsecurity','military personnel', 
  'military strength','cuba','trade agreements', 'putin',
8.'healthcare','obamacare','healthcare (including medicare)',
9.'immigration','wall','immigration reform', 'border', ' borders','border security','bordersecurity', 'latinos',
10.'iran','iran deal',
11.'iraq','wmds and iraq',
12.'isis','radical islam',
13.'jobs','jobs and economy', 'auto bailout','jobs and wages', 'small business', 'exporting jobs', 'economy',
   'trade deficit','currency', 'wall street', 'corporate america', 'deficit', 'taxes', 'irs',
   'crumbling infrastructure'
14.'lgbt issues','gay marriage',
15.'none','none of the above',
16.'poverty', 'minimum wage',
17.'racial equality','racial issues','black incarceration','racism', 'klu klux klan',
18.'religion', 'personal religion', 'religious liberty',
19.'supreme court','supreme court appointments','supremecourt',
20.'syria', 'syrian refugees',
21.'trump', 'trump cronyism','trump failed business','trump flip-flop on issues',
  'trump funding candidates','trump greed','trump offshore jobs','trump racism','trump university', 'trump wealth',
22.'zika','zikavirus'


In [84]:
T_new.ix[T_new['topic'] == 'clintons emails','topic'] = 'clinton emails' #1
T_new.ix[T_new['topic'].isin( ['cnndebate','cnntownhall']),'topic'] = 'cnn' #2
T_new.ix[T_new['topic'].isin( ['college tuition', 'education','public education', 'schools']),'topic'] = 'college' #3
T_new.ix[T_new['topic'].isin( ['fracking','climate change']),'topic'] = 'environment' #4
T_new.ix[T_new['topic'] == 'familyleave','topic'] = 'family leave'#5
T_new.ix[T_new['topic'].isin( ['flint water','flintwater']),'topic'] = 'flintwatercrisis' #6
T_new.ix[T_new['topic'].isin( ['foreign aid', 'north korea', 'homelandsecurity','military personnel', 
  'military strength','cuba','trade agreements', 'putin']),'topic'] = 'foreign policy' #7
T_new.ix[T_new['topic'].isin( ['healthcare (including medicare)','obamacare']),'topic'] = 'healthcare' #8
T_new.ix[T_new['topic'].isin( ['immigration','wall','immigration reform', 'border', ' borders','border security',
                               'bordersecurity', 'latinos']),'topic'] = 'immigration' #9
T_new.ix[T_new['topic'] == 'iran deal','topic'] = 'iran'  #10
T_new.ix[T_new['topic'] == 'wmds and iraq','topic'] = 'iraq'  #11
T_new.ix[T_new['topic'] == 'radical islam','topic'] = 'isis' #12
T_new.ix[T_new['topic'].isin( ['jobs', 'auto bailout','jobs and wages', 'small business', 'exporting jobs', 'economy',
   'trade deficit','currency', 'wall street', 'corporate america', 'deficit', 'taxes', 'irs',
   'crumbling infrastructure']),'topic'] = 'jobs and economy' #13
T_new.ix[T_new['topic'] == 'gay marriage','topic'] = 'lgbt issues'  #14
T_new.ix[T_new['topic'] == 'none of the above','topic'] = 'none' #15
T_new.ix[T_new['topic'] == 'minimum wage','topic'] = 'poverty' #16
T_new.ix[T_new['topic'].isin( ['racial equality','black incarceration','racism', 'klu klux klan']),'topic'] = 'racial issues' #17
T_new.ix[T_new['topic'].isin( ['personal religion', 'religious liberty']),'topic'] = 'religion' #18
T_new.ix[T_new['topic'].isin( ['supreme court appointments','supremecourt']),'topic'] = 'supreme court' #19
T_new.ix[T_new['topic'] == 'syrian refugees','topic'] = 'syria' #20
T_new.ix[T_new['topic'].isin( ['trump', 'trump cronyism','trump failed business','trump flip-flop on issues',
  'trump funding candidates','trump greed','trump offshore jobs','trump racism','trump university',
                               'trump wealth']),'topic'] = 'none' #21

T_new.ix[T_new['topic'] == 'zika','topic'] = 'zikavirus'   #22

T_new.ix[T_new['topic'] == 'hillary','topic'] = 'none' 

In [85]:
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

['abortion',
 'afghanistan',
 'benghazi',
 'bidding procedures',
 'bipartisanship',
 'buzzfeed tapes',
 'campaign finance',
 'candidate behavior',
 'character',
 'clinton emails',
 'clintons wall street speeches',
 'cnn',
 'college',
 'common core',
 'community',
 'conservatism',
 'cutting government',
 'deathpenalty',
 'delegate count',
 'environment',
 'experience',
 'family leave',
 'flintwatercrisis',
 'foreign policy',
 'fox news or moderators',
 'gop',
 'gun control',
 'healthcare',
 'immigration',
 'insiders vs outsiders',
 'iran',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'lgbt issues',
 'marijuana',
 'michigan primary',
 'none',
 'obama',
 'poverty',
 'prosperity',
 'puerto rico',
 'racial issues',
 'religion',
 'role of government',
 'romney',
 'ronaldreagan',
 'rubio absenteeism',
 'snowden',
 'social security',
 'socialism',
 'staff',
 'supreme court',
 'syria',
 'taped discussion',
 'teachers unions',
 'torture',
 'trustworthiness',
 'veterans',
 "women's issues (

In [86]:
# remove intro closing nan and flipflopping 

T_new =T_new[~T_new['topic'].isin(['intro','flipflopping','closing'])]
T_new.shape

(6338, 2)

In [87]:
# set all nans in topic to none
T_new = T_new[~T_new['topic'].isnull()]

In [88]:
T_new['topic'].value_counts()

none                                    1815
religion                                 419
racial issues                            384
women's issues (not abortion though)     362
jobs and economy                         300
abortion                                 293
immigration                              287
cnn                                      228
fox news or moderators                   188
socialism                                145
healthcare                               140
iran                                     132
gun control                              130
lgbt issues                              128
flintwatercrisis                         125
isis                                     106
israel                                    92
benghazi                                  90
syria                                     85
foreign policy                            83
poverty                                   74
veterans                                  73
supreme co

In [89]:
# Sample the data so as to get equal numbers of each class.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(T_new['ascii'],T_new['topic'], test_size=0.3, random_state=42)


In [90]:
    
print len(X_train), len(X_test) , len(y_train) , len(y_test)

4436 1902 4436 1902


In [91]:
stop_words = pd.read_table('/Users/preetijavaji/Documents/DSE/capstone/stop_words.org',names='words')
stop_words_list = list(stop_words['w'])
from sklearn.feature_extraction.text import CountVectorizer
vectorizer   =  CountVectorizer(min_df = 1,stop_words = stop_words_list, ngram_range = (1,2))
X_f_train = vectorizer.fit_transform(X_train)

In [92]:
classifiers = []
#classifiers.append( svm.SVC() )
classifiers.append( OneVsOneClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)) )

#classifier_1 = OneVsOneClassifier(svm.SVC())
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

cv = 10  # Number of crossvalidation folds to use.
scores = []

for classifier in classifiers:
    scores.append( cross_validation.cross_val_score(classifier, X_f_train, y_train, cv=cv) )
    
for score in scores:
    print score
    print score.mean()
    print

print 'done'

[ 0.78451883  0.76923077  0.7978022   0.79241071  0.80995475  0.80504587
  0.80414747  0.81074766  0.81176471  0.82464455]
0.80102675171

[ 0.82217573  0.78846154  0.81538462  0.83258929  0.84162896  0.83715596
  0.82258065  0.83411215  0.83294118  0.84834123]
0.827537129775

[ 0.75941423  0.73076923  0.78901099  0.765625    0.79864253  0.76146789
  0.78341014  0.78037383  0.79058824  0.80331754]
0.776261961043

done


In [93]:
XT_f = vectorizer.transform(X_test)
predicteds = []
for classifier in classifiers:
    classifier.fit(X_f_train, y_train)
    yy = classifier.predict(XT_f)
    print yy.shape
    predicteds.append( classifier.predict(XT_f))


(1902,)
(1902,)
(1902,)


In [94]:
# Apply classifiers to the whole (not sampled) dataset and look at the confusion matrix and classification report for each.

# Format the whole dataset.
X_all = T_new['ascii'].values
y_all = T_new['topic'].values


# Create feature vectors from the tweets. Use the same feature vector as above.
X_featurized_all = vectorizer.transform(X_all)

# Generate predictions using the classifier trained on the equally sampled data.
predicteds = []
for classifier in classifiers:
    predicteds.append( classifier.predict(X_featurized_all) )
    
for y_pred_all in predicteds:
    print accuracy_score(y_all, y_pred_all)
    #print '   Neg ', 'Neu ', 'Pos'
    print confusion_matrix(y_all, y_pred_all)
    print classification_report(y_all, y_pred_all)
    print


0.929315241401
[[257   0   0 ...,   0  16   0]
 [  0   3   0 ...,   0   0   0]
 [  0   0  87 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,  70   0   0]
 [ 11   0   0 ...,   0 323   0]
 [  0   0   0 ...,   0   0  31]]
             precision    recall  f1-score   support

   abortion       0.91      0.88      0.89       293
afghanistan       1.00      1.00      1.00         3
   benghazi       0.99      0.97      0.98        90
bidding procedures       1.00      0.83      0.91         6
bipartisanship       1.00      1.00      1.00         2
buzzfeed tapes       0.00      0.00      0.00         1
campaign finance       1.00      0.73      0.84        11
candidate behavior       1.00      0.50      0.67         4
  character       1.00      0.75      0.86         4
clinton emails       0.95      0.90      0.93        42
clintons wall street speeches       1.00      0.80      0.89        10
        cnn       0.95      0.96      0.95       228
    college       1.00      0.84      0.91       

In [95]:
# read the tweet sample into a df
Tweet_df = pd.read_table('Tweet_g6.csv',sep=',',names=['tweetid','tweet','c3','c4','c5','c6','c7','c8'])
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,NaN,"""Six N.J. newspapers call on Christie to resign https://t.co/f7RQzkYXHA """,Tweet_2016_3_2,1456873900000,704806338304602113,NaN,Tue Mar 01 23:11:40 +0000 2016,167
1,7.048063e+17,"""RT @JillDLawrence: Six N.J. newspapers call on Christie to resign https://t.co/f7RQzkYXHA """,Tweet_2016_3_2,1456876800051,704818500418871296,NaN,Wed Mar 02 00:00:00 +0000 2016,0
2,NaN,"""I honestly cannot understand how Trump has so many supporters. """,Tweet_2016_3_2,1456876800025,704818500309749760,NaN,Wed Mar 02 00:00:00 +0000 2016,0
3,NaN,"""Nikki Haley Responds to Latest Trump Insult With This Three-Word Tweet https://t.co/hF7ZNDeTN7 via @theblaze """,Tweet_2016_3_2,1456876800064,704818500473212928,NaN,Wed Mar 02 00:00:00 +0000 2016,0
4,NaN,"""Trump votes changed to Rubio in TX - Listen to KLBJ audio, callers reporting this! https://t.co/g47l4mQE5z #voterfraud """,Tweet_2016_3_2,1456857504000,704737569540411393,NaN,Tue Mar 01 18:38:24 +0000 2016,16
5,7.047376e+17,"""RT @JennInTexas88: Trump votes changed to Rubio in TX - Listen to KLBJ audio, callers reporting this! https://t.co/g47l4mQE5z #voterfraud """,Tweet_2016_3_2,1456876800171,704818500922155008,NaN,Wed Mar 02 00:00:00 +0000 2016,0
6,NaN,"""If this all comes down to Hillary and Trump and no Bernie I'm going to cry in a box for a year """,Tweet_2016_3_2,1456876800207,704818501073158144,NaN,Wed Mar 02 00:00:00 +0000 2016,0
7,NaN,"""@KarrattiPaul @DrMartyFox @GregAbbott_TX @GOP Call the GOP offices right now! """,Tweet_2016_3_2,1456874863000,704810379705102336,7.048012e+17,Tue Mar 01 23:27:43 +0000 2016,0
8,7.048104e+17,"""RT @sandi_hemming: @KarrattiPaul @DrMartyFox @GregAbbott_TX @GOP Call the GOP offices right now! """,Tweet_2016_3_2,1456876800230,704818501169577984,NaN,Wed Mar 02 00:00:00 +0000 2016,0
9,NaN,"""@danscavino @clewandowski Report: Trump Supporters in Texas See Votes Switched to Rubio https://t.co/1NQ2vXBsrK via @realalexjones """,Tweet_2016_3_2,1456860456000,704749951683682304,NaN,Tue Mar 01 19:27:36 +0000 2016,27


In [96]:
# Convert to ascii
Tweet_df['ascii'] = Tweet_df['tweet'].apply(to_ascii)

# Remove hashtags
#df['ascii'] = df['ascii'].str.replace(r"#([A-Za-z0-9_]+)", " ")

# Remove handles
#df['ascii'] = df['ascii'].str.replace(r"@([A-Za-z0-9_]+)", " ")

# Remove URLs
#df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", " ")
# Replace URLs with "URL"
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"(['';:@%#()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"([;:@%#()\+\*\"\…\“\”])", "")

# Remove eol symbols
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbolst
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
Tweet_df['ascii'] = Tweet_df['ascii'].str.lower()


Tweet_df[['tweet', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)


,tweet,ascii
0,"""Six N.J. newspapers call on Christie to resign https://t.co/f7RQzkYXHA """,six n.j. newspapers call on christie to resign url
1,"""RT @JillDLawrence: Six N.J. newspapers call on Christie to resign https://t.co/f7RQzkYXHA """,rt @jilldlawrence six n.j. newspapers call on christie to resign url
2,"""I honestly cannot understand how Trump has so many supporters. """,i honestly cannot understand how trump has so many supporters.


In [97]:
X = Tweet_df['ascii'].values

#train_vectors = vectorizer.fit_transform(train_data)
#test_vectors = vectorizer.transform(test_data)
X_featurized = vectorizer.transform(X)
        
print X_featurized.shape
print 'done'

(10000, 40008)
done


In [98]:
predicted_new = []
for classifier in classifiers:
    predicted_new.append( classifier.predict(X_featurized) )

In [99]:
Tweet_df['topic'] = predicted_new[0]

In [100]:
from collections import Counter

Keys = Counter(predicted_new[0]).keys() # equals to list(set(words))
values =  Counter(predicted_new[0]).values() # counts the elements' frequency
for k,v in zip(Keys,values):
    print k,':',v

racial issues : 376
fox news or moderators : 125
ronaldreagan : 1
marijuana : 3
college : 7
family leave : 2
lgbt issues : 4
environment : 7
religion : 31
clinton emails : 4
cnn : 574
veterans : 5
gop : 32
benghazi : 16
israel : 8
iran : 8
immigration : 59
poverty : 6
foreign policy : 1
iraq : 7
obama : 85
none : 7411
isis : 5
gun control : 32
jobs and economy : 88
abortion : 17
socialism : 983
conservatism : 10
healthcare : 10
syria : 2
women's issues (not abortion though) : 81


In [101]:
Tweet_df[Tweet_df['topic'] == "racial issues"]['tweet']

19              "Shorter Michael Moore: ignore the southern black voters that the modern Democratic Party all but exists to help  https://t.co/Q8sMPnmBQl "
20          "RT @Nate_Cohn: Shorter Michael Moore: ignore the southern black voters that the modern Democratic Party all but exists to help  https://t.c… "
24          "RT @KLSouth: Newt Gingrich: @GOP Establishment ‘Denying Reality’ Should ‘Question Their Failure’ -  https://t.co/sVi8gQf2YF https://t.co/Li… "
51          "RT @FrankLuntz: Bernie supporters say Bill Clinton blocked a Boston polling place for several hours today with his motorcade.  https://t.co… "
61                 "What to expect as presidential candidates race for delegates? Morgan Stanley's Government Relations examines: https://t.co/1wawTFkqJ6 "
62                                                      "GOP 2008: I'm not a racist GOP 2012: I'm not a racist, but GOP 2016: Proudly endorsed by the KKK "
63                                      "RT @linnyitssn: GOP 200

In [102]:
import pickle
# save model in a pickle file
output = open('mymodel_g6.pkl', 'wb')
pickle.dump(classifiers[0],output)
output.close()

In [103]:
vec_file = open('myvec_g6.pkl','wb')
pickle.dump(vectorizer,vec_file)
vec_file.close()